<center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks&nbsp;&para;&para;">unSkript Runbooks</h1>
<div class="alert alert-block alert-success"><strong>To modify old generation RDS instances to a given instance class in AWS using unSkript actions.</strong></div>
</center><center>
<h2 id="Modify-RDS-Instances-Using-Previous-Gen-Instance-Types-in-AWS">AWS Update RDS Instances from Old to New Generation</h2>
</center>
<h1 id="Steps-Overview&para;&para;">Steps Overview</h1>
<ol>
<li>AWS Get Older Generation RDS Instances</li>
<li>Modify the DB Instance Class</li>
</ol>

In [6]:
if not rds_instance_ids and rds_instance_type and region:
    raise SystemExit("Provide rds_instance_ids!")
if not rds_instance_type:
    raise SystemExit("Provide rds_instance_type!")
if not region and rds_instance_type and rds_instance_ids:
    raise SystemExit("Provide region!")
if region == None:
    region = ""

<h3 id="Filter-Unattached-EBS-Volumes">AWS Get Older Generation RDS Instances</h3>
<p>Here we will use unSkript&nbsp;<strong>AWS Get Older Generation RDS Instances</strong> action. This action filters all the rds instances from the given region and returns a list of all the older generation instance-type instances. It will execute if the <code>rds_instance_ids</code>&nbsp;parameter is not passed.</p>
<blockquote>
<p>Input parameters: <code>region</code></p>
</blockquote>
<blockquote>
<p>Output variable: <code>old_gen_rds_instances</code></p>
</blockquote>

In [ ]:
##
##  Copyright (c) 2023 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Optional, Tuple
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
from unskript.connectors.aws import aws_get_paginator
import pprint


from beartype import beartype
@beartype
def aws_get_older_generation_rds_instances_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def is_previous_gen_instance(instance_type):
    previous_gen_instance_types = ['db.m1', 'db.m2', 'db.t1']
    for prev_gen_type in previous_gen_instance_types:
        if instance_type.startswith(prev_gen_type):
            return True
    return False


@beartype
def aws_get_older_generation_rds_instances(handle, region: str = "") -> Tuple:
    """aws_get_older_generation_rds_instances Gets all older generation RDS DB instances

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type region: string
        :param region: Optional, Region of the RDS.

        :rtype: Status, List of old RDS Instances
    """
    result = []
    all_regions = [region]
    if not region:
        all_regions = aws_list_all_regions(handle)
    for reg in all_regions:
        try:
            ec2Client = handle.client('rds', region_name=reg)
            response = aws_get_paginator(ec2Client, "describe_db_instances", "DBInstances")
            for db in response:
                instance_type = ".".join(db['DBInstanceClass'].split(".", 2)[:2])
                response = is_previous_gen_instance(instance_type)
                if response:
                    db_instance_dict = {}
                    db_instance_dict["region"] = reg
                    db_instance_dict["instance"] = db['DBInstanceIdentifier']
                    result.append(db_instance_dict)
        except Exception:
            pass

    if len(result) != 0:
        return (False, result)
    else:
        return (True, None)


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "region"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not rds_instance_ids",
    "condition_result": true
    }''')

task.configure(outputName="old_gen_rds_instances")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_older_generation_rds_instances, lego_printer=aws_get_older_generation_rds_instances_printer, hdl=hdl, args=args)

<h3 id="Modify-Step-1-Output">Modify Step-1 Output<a class="jp-InternalAnchorLink" href="#Modify-Step-1-Output" target="_self">&para;</a></h3>
<p>In this action, we modify the output from step 1 and return a list of aws cli command for the older generation RDS instances.</p>
<blockquote>
<p>Output variable: <code>older_rds_instances</code></p>
</blockquote>

In [ ]:
older_rds_instances = []
try:
    if old_gen_rds_instances[0] == False:
        for instance in old_gen_rds_instances[1]:
            instance['instance_type'] = rds_instance_type
            command = "aws rds modify-db-instance --db-instance-identifier " + instance['instance'] + " --db-instance-class " + instance['instance_type'] + " --region " + instance['region'] + " --apply-immediately"
            older_rds_instances.append(command)
except Exception as e:
    for i in rds_instance_ids:
        command = "aws rds modify-db-instance --db-instance-identifier " + i + " --db-instance-class " + rds_instance_type + " --region " + region + " --apply-immediately"
        older_rds_instances.append(command)
    else:
        raise Exception(e)

<p><strong>Modify DB Instance Class</strong></p>
<p>In this action, we pass an aws cli command to modify the RDS instance class.</p>
<blockquote>
<p>Input parameters: <code>aws_command</code></p>
</blockquote>
<blockquote>
<p>Output variable: <code>modified_output</code></p>
</blockquote>

In [ ]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
import pprint


from beartype import beartype
@beartype
def aws_execute_cli_command_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_execute_cli_command(handle, aws_command: str) -> str:

    result = handle.aws_cli_command(aws_command)
    if result is None or result.returncode != 0:
        print(
            f"Error while executing command ({aws_command}): {result}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "aws_command": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "older_rds_instances",
    "iter_parameter": "aws_command"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(older_rds_instances)!=0",
    "condition_result": true
    }''')
task.configure(outputName="modified_output")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_execute_cli_command, lego_printer=aws_execute_cli_command_printer, hdl=hdl, args=args)

<h3 id="Conclusion">Conclusion</h3>
<p>In this Runbook, we demonstrated the use of unSkript's AWS actions to get RDS instances with old generation and modify them to the new given instance class. To view the full platform capabunscriptedof unSkript please visit <a href="https://us.app.unskript.io" target="_blank" rel="noopener">https://us.app.unskript.io</a></p>